# Assignment 7

### Part A

In [21]:
import numpy as np
import pandas as pd

Consider the following way of representing a portfolio. Each row in the DataFrame represents one security, with information given about the security's beta, volatility, and amount invested in that security. Note that there is not guaranteed to be 2 securities in the portfolio.

In [22]:
portfolio = pd.DataFrame(data = {"Beta": [1.2, 1.4], 
                                 "Volatility": [0.25, 0.4],
                                 "Amount Invested": [4000, 6000] })



portfolio

Beta  Volatility  Amount Invested
0   1.2        0.25             4000
1   1.4        0.40             6000

Write the function ```expected_return```, which calculates the expected return of the portfolio ```P```. There is a risk free asset with risk free rate ```rf```, and the market risk premium is ```mkt_rp```. 

expected return: expected_return(portfolio, 0.02, 0.05) should yield 0.086. 

In [23]:
def expected_return(P, rf, mkt_rp):
    
    # Calculate the expected return for each security
    P['Expected Return'] = rf + P['Beta'] * mkt_rp
    
    # Calculate the total amount invested in the portfolio
    total_invested = P['Amount Invested'].sum()
    
    # Calculate the weight of each security in the portfolio
    P['Weight'] = P['Amount Invested'] / total_invested
    
    # Calculate the expected return of the portfolio
    portfolio_return = (P['Expected Return'] * P['Weight']).sum()
    
    return portfolio_return

Write the function ```volatility```, which calculates the volatility of the portfolio. You can assume that the correlation between all (different) pairs of stocks in the portfolio is ```cor```.

expected return: volatility(portfolio, -0.2) should yield 0.2408318915758459

In [24]:
def volatility(P, cor):
    
     # Calculate the weights of the securities in the portfolio
    total_invested = P['Amount Invested'].sum()
    P['Weight'] = P['Amount Invested'] / total_invested
    
    # Initialize portfolio variance
    portfolio_variance = 0
    
    # Calculate the variance and covariance components for the portfolio variance
    for i in range(len(P)):
        for j in range(len(P)):
            if i == j:
                # This is a variance component
                portfolio_variance += (P.loc[i, 'Weight'] ** 2) * (P.loc[i, 'Volatility'] ** 2)
            else:
                # This is a covariance component
                portfolio_variance += (P.loc[i, 'Weight'] * P.loc[j, 'Weight'] *
                                       P.loc[i, 'Volatility'] * P.loc[j, 'Volatility'] * cor)
    
    # The portfolio volatility is the square root of the portfolio variance
    portfolio_volatility = np.sqrt(portfolio_variance)
    
    return portfolio_volatility

Suppose there is a risk-free asset with return of ```rf```, the market risk premium is ```mkt_rp```, and the volatility of returns on the market is ```mkt_vol```.

Write the function ```expected_return2```, which creates a portfolio with volatility ```vol``` using (only) the risk free asset and the market portfolio, and calculates the expected return of that portfolio.

expected return: expected_return2(0.02, 0.05, 0.16, 0.2408) should yield 0.09525

In [25]:
def expected_return2(rf, mkt_rp, mkt_vol, vol):
    # Calculate the Sharpe Ratio using the market risk premium and market volatility
    sharpe_ratio = mkt_rp / mkt_vol
    
    # Calculate the expected return using the Sharpe Ratio and the desired portfolio volatility
    expected_return_portfolio = rf + sharpe_ratio * vol
    
    return expected_return_portfolio

### Part B

Write the function ```CAPM_beta```, which will consume a **filepath** ```return_path```, (formatted as the provided "monthlycsrp_mod7.csv"), and a **filepath** `factors_path` (formatted as the provided "F-F_Research_Data_Factors.CSV"). This function will return a DataFrame formatted like the one supplied. Each row in the returned DataFrame should correspond with 1 CUSIP, its expected return, and its CAPM-beta.

Please note that monthlycsrp_mod7.csv's return data are in decimals. "F-F_Research_Data_Factors.CSV" has data in percentages. More details of "F-F_Research_Data_Factors.CSV" are available on https://mba.tuck.dartmouth.edu/pages/faculty/ken.french/Data_Library/f-f_factors.html. 

Also, note that firm i's CAPM-beta is calculated as Cov(R_i-Rf, R_mkt – Rf)/Var(R_mkt-Rf). 

expected return: it should return output of 3225 rows × 3 columns

In [40]:
def CAPM_beta(return_path, factors_path):
    
    # Read the returns and factors data
    returns = pd.read_csv(return_path)
    factors = pd.read_csv(factors_path)
    
    # Convert the market returns and risk-free rate from percentages to decimals
    factors['Mkt-RF'] = factors['Mkt-RF'] / 100
    factors['RF'] = factors['RF'] / 100
    
    # Calculate the expected market return (Mkt-RF + RF)
    factors['Mkt'] = factors['Mkt-RF'] + factors['RF']
    
    # Merge the returns and factors DataFrames based on the date
    returns['date'] = returns['year'].astype(str) + returns['month'].astype(str).str.zfill(2)
    factors['date'] = factors['date'].astype(str)
    merged_data = returns.merge(factors, on='date')
    
    # Calculate excess returns for the securities and the market
    merged_data['Excess_RET'] = merged_data['RET'] - merged_data['RF']
    merged_data['Excess_Mkt'] = merged_data['Mkt-RF'] - merged_data['RF']
    
    # Group by CUSIP and calculate expected return and CAPM beta for each CUSIP
    output = merged_data.groupby('CUSIP').apply(lambda x: pd.Series({
        'EXP_RETURN': x['RET'].mean(),
        'BETA': x['Excess_RET'].cov(x['Excess_Mkt']) / x['Excess_Mkt'].var()
    })).reset_index()
    
    return output

         CUSIP  EXP_RETURN      BETA
0     00081T10    0.016252  0.029514
1     00108M10    0.011101  0.015654
2     00130H10    0.016152  0.017528
3     00163U10    0.015538  0.014027
4     00182C10    0.014772  0.017551
...        ...         ...       ...
3220   9959910    0.009933  0.015371
3221   9972410    0.012866  0.016057
3222   9972510    0.017177  0.019588
3223   9984910    0.009006  0.018237
3224   9985510    0.015699  0.019605

[3225 rows x 3 columns]


Write the function ```slope```, which will return the slope of the security market line. This function will take in a **DataFrame** ```data```, whose format is that of the output of ```CAPM_beta```. You may find the function `np.polyfit` useful.

In [45]:
def slope(data):
    
    betas = data['BETA']
    exp_returns = data['EXP_RETURN']

    # Using np.polyfit to fit a first-degree polynomial (a line) to the data, which returns the slope and intercept
    coefficients = np.polyfit(betas, exp_returns, 1)
    
    slope = coefficients[0]

    return slope

0.004773236586034263
